<a href="https://colab.research.google.com/github/Jeevesh8/arg_mining/blob/main/experiments/long_context_am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies

In [ ]:
%%capture
#if running on colab, install below 4
#!git clone https://github.com/Jeevesh8/arg_mining
#!pip install transformers
#!pip install seqeval datasets allennlp
#!pip install flax

#if connected to local runtime, run the next command too
#pip install bs4 tensorflow torch 



*   Update ``arg_mining/datasets/cmv_modes/configs.py`` as per your requirements, all experiments considered till now, set ``batch_size`` to 2, and all other variables with their default value.



In [20]:
#Run to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Load Metric

In [21]:
from datasets import load_metric
metric = load_metric('seqeval')

### Define & Load Tokenizer, Model, Dataset

In [22]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [23]:
device

device(type='cuda', index=0)

In [24]:
model_version = 'allenai/longformer-base-4096'

In [25]:
%%capture
from transformers import LongformerTokenizer, AutoModel
tokenizer = LongformerTokenizer.from_pretrained(model_version)
transformer_model = AutoModel.from_pretrained(model_version).to(device)

In [26]:
with open('./Discourse_Markers.txt') as f:
    discourse_markers = [dm.strip() for dm in f.readlines()]

In [27]:
%%capture
from arg_mining.datasets.cmv_modes import load_dataset, data_config

tokenizer.add_tokens(data_config["special_tokens"])

transformer_model.resize_token_embeddings(len(tokenizer))

def get_datasets():
    train_dataset, valid_dataset, test_dataset = load_dataset(tokenizer=tokenizer,
                                                              train_sz=80,
                                                              test_sz=20,
                                                              mask_tokens=discourse_markers)
    return train_dataset, valid_dataset, test_dataset

### Define layers for a Linear-Chain-CRF

In [28]:
import torch.nn as nn

from allennlp.modules.conditional_random_field import ConditionalRandomField as crf

ac_dict = data_config["arg_components"]

allowed_transitions =([(ac_dict["B-C"], ac_dict["I-C"]), 
                       (ac_dict["B-P"], ac_dict["I-P"])] + 
                      [(ac_dict["I-C"], ac_dict[ct]) 
                        for ct in ["I-C", "B-C", "B-P", "O"]] +
                      [(ac_dict["I-P"], ac_dict[ct]) 
                        for ct in ["I-P", "B-C", "B-P", "O"]] +
                      [(ac_dict["O"], ac_dict[ct]) 
                        for ct in ["O", "B-C", "B-P"]])
                    
linear_layer = nn.Linear(transformer_model.config.hidden_size,
                         len(ac_dict)).to(device)

crf_layer = crf(num_tags=len(ac_dict),
                constraints=allowed_transitions,
                include_start_end_transitions=False).to(device)

cross_entropy_layer = nn.CrossEntropyLoss(weight=torch.log(torch.tensor([3.8087, 54.6686, 3.4161, 45.3662, 2.4729], 
                                                                        device=device)), reduction='none')

### Global Attention Mask Utility for Longformer

In [29]:
import numpy as np
from threading import Lock
MUTEX = Lock()

def get_global_attention_mask(tokenized_threads: np.ndarray) -> np.ndarray:
    """Returns an attention mask, with 1 where there are [USER{i}] tokens and 
    0 elsewhere.
    """
    mask = np.zeros_like(tokenized_threads)
    for user_token in ["UNU"]+[f"[USER{i}]" for i in range(data_config["max_users"])]:
        MUTEX.acquire()
        try:
            user_token_id = tokenizer.encode(user_token)[1:-1]
        finally:
            MUTEX.release()
        mask = np.where(tokenized_threads==user_token_id, 1, mask)
    mask = np.where(tokenized_threads==tokenizer.mask_token_id, 1, mask)        #Make discourse markers global
    return np.array(mask, dtype=bool)

### Loss and Prediction Function

In [30]:
from typing import Tuple

In [31]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
            preds: bool=False, cross_entropy: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        preds:  If True, returns a List(of batch_size size) of Tuples of form 
                (tag_sequence, viterbi_score) where the tag_sequence is the 
                viterbi-decoded sequence, for the corresponding sample in the batch.
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, comp_type_labels, global_attention_mask = batch
    
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,
                               attention_mask=pad_mask,
                               global_attention_mask=global_attention_mask).last_hidden_state)
    
    if preds:
        return crf_layer.viterbi_tags(logits, pad_mask)
    
    log_likelihood = crf_layer(logits, comp_type_labels, pad_mask)
    
    if cross_entropy:
        logits = logits.reshape(-1, logits.shape[-1])
        
        pad_mask, comp_type_labels = pad_mask.reshape(-1), comp_type_labels.reshape(-1)
        
        ce_loss = torch.sum(pad_mask*cross_entropy_layer(logits, comp_type_labels))
        
        return ce_loss - log_likelihood

    return -log_likelihood

### Define optimizer

In [32]:
from itertools import chain

import torch.optim as optim

optimizer = optim.Adam(params = chain(transformer_model.parameters(),
                                      linear_layer.parameters(),
                                      crf_layer.parameters()),
                       lr = 2e-5,)

### Training And Evaluation Loops

In [33]:
def train(dataset):
    accumulate_over = 4
    
    optimizer.zero_grad()

    for i, (tokenized_threads, masked_threads, comp_type_labels, _ ) in enumerate(dataset):
        global_attention_mask = torch.tensor(get_global_attention_mask(masked_threads),
                                             device=device, dtype=torch.int32)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device, dtype=torch.long)

        loss = compute((masked_threads, 
                        comp_type_labels, 
                        global_attention_mask))/data_config["batch_size"]
        
        print("Loss: ", loss)
        loss.backward()
        
        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
    
    optimizer.step()

In [34]:
def evaluate(dataset, metric):
    
    int_to_labels = {v:k for k, v in ac_dict.items()}
    
    for tokenized_threads, masked_threads, comp_type_labels, _ in dataset:
        
        global_attention_mask = torch.tensor(get_global_attention_mask(masked_threads), 
                                             device=device)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device)

        preds = compute((masked_threads,
                         comp_type_labels,
                         global_attention_mask),
                        preds=True)
        
        lengths = torch.sum(torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0), 
                            axis=-1)
        
        preds = [ [int_to_labels[pred] for pred in pred[0][:lengths[i]]]
                  for i, pred in enumerate(preds)
                ]
        
        refs = [ [int_to_labels[ref] for ref in labels[:lengths[i]]]
                 for i, labels in enumerate(comp_type_labels.cpu().tolist())
               ]
        
        metric.add_batch(predictions=preds, 
                         references=refs)
    
    print(metric.compute())

### Final Training

In [35]:
n_epochs = 35

In [36]:
for epoch in range(n_epochs):
    print(f"------------EPOCH {epoch+1}---------------")
    train_dataset, _, test_dataset = get_datasets()
    train(train_dataset)
    evaluate(test_dataset, metric)

------------EPOCH 1---------------


fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(15906.0625, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15876.7637, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(16095.1572, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15780.4287, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15138.1533, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15131.2686, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15070.5254, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14919.9160, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14230.7051, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14387.2236, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14541.8105, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14523.3809, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13445.7793, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13439.3535, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13835.3887, device='cuda:0', grad_fn=<DivBackwar

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(3698.1750, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3010.5999, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3792.6653, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3191.5071, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2107.5073, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2182.1240, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2562.9373, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3067.6360, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2383.0564, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2510.3882, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2396.1143, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3198.0974, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2585.1382, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3644.6587, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3945.9744, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  ten

...

Loss:  tensor(155.8527, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(99.3855, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(102.7512, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(63.1833, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(58.6584, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(51.7055, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(60.7513, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(64.0932, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(42.3113, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(35.6412, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(68.5385, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(111.6991, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(82.9142, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(153.6952, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(176.7389, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(103.5699, device='cud

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(140.1010, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(87.5412, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(92.0046, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(57.4772, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(49.2518, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(49.4184, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(55.1583, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(54.9100, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(38.0780, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(32.3045, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(61.5280, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(95.9852, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(75.3524, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(139.3478, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(158.5520, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(92.3805, device='cuda:0

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(126.1632, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(81.8223, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(80.9559, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(51.5541, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(44.2213, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(45.0510, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(50.5465, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(49.1138, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(32.7777, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(28.0559, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(57.7866, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(81.4146, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(69.1635, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(124.8666, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(140.9236, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(84.9452, device='cuda:0